## Latent Dirichlet Allocation

TODO, Explanation of how LDA works to generate vectors from sentences....

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df_variants = pd.read_csv('../data/training_variants.csv',index_col=0)

In [3]:
tokenized_data = pd.read_pickle('../data/new_tokens.pk')
tokenized_data.head()
df = pd.merge(df_variants,tokenized_data,left_index=True,right_index=True)
df.head()

,Gene,Variation,Class,text,processed
ID,,,,,
0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...,"[cyclin-depend, kinas, cdk, regul, varieti, fu..."
1,CBL,W802*,2,Abstract Background Non-small cell lung canc...,"[abstract, background, non-smal, cell, lung, c..."
2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...,"[abstract, background, non-smal, cell, lung, c..."
3,CBL,N454D,3,Recent evidence has demonstrated that acquired...,"[recent, evid, demonstr, acquir, uniparent, di..."
4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...,"[oncogen, mutat, monomer, casita, b-lineag, ly..."


In [4]:
def return_same(ls):
    return ls
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
pipe = Pipeline([('encoder',CountVectorizer(analyzer=return_same,max_df=.9,
                                   min_df=50)),('lda',LatentDirichletAllocation(n_topics=50, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,n_jobs=-1))])

In [5]:
X = df.processed.values
x_tf = pipe.fit_transform(X)
x_tf[:10]

array([[  4.21371707e-02,   8.37871806e-06,   4.12374461e-01,
          8.37871806e-06,   8.37871806e-06,   8.37871806e-06,
          8.37871806e-06,   8.37871806e-06,   8.37871806e-06,
          8.37871806e-06,   8.37871806e-06,   8.37871806e-06,
          8.37871806e-06,   8.37871806e-06,   8.37871806e-06,
          8.37871806e-06,   1.17868180e-03,   8.37871806e-06,
          8.37871806e-06,   6.79067330e-02,   1.43459682e-02,
          8.37871806e-06,   5.31150102e-02,   8.37871806e-06,
          8.37871806e-06,   1.85187028e-01,   8.37871806e-06,
          8.37871806e-06,   7.49427590e-02,   8.37871806e-06,
          7.16238077e-03,   8.37871806e-06,   8.37871806e-06,
          8.37871806e-06,   8.37871806e-06,   8.37871806e-06,
          8.89286896e-02,   8.37871806e-06,   8.37871806e-06,
          8.37871806e-06,   8.37871806e-06,   1.29646361e-02,
          8.37871806e-06,   8.37871806e-06,   8.37871806e-06,
          8.37871806e-06,   8.37871806e-06,   8.37871806e-06,
        

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
-cross_val_score(SVC(probability=True),x_tf,df.Class,scoring='neg_log_loss')

array([ 1.72749109,  1.77404463,  1.96317633])

In [7]:
from xgboost import XGBClassifier
-cross_val_score(LogisticRegression(),x_tf,df.Class,scoring='neg_log_loss')

array([ 1.70167561,  1.69297151,  1.6944934 ])

In [12]:
from sklearn.ensemble import RandomForestClassifier
-cross_val_score(RandomForestClassifier(n_estimators=1000,n_jobs=-1),x_tf,df.Class,scoring='neg_log_loss')

array([ 2.93187115,  2.53994374,  2.44306664])

In [13]:
xgbc = XGBClassifier(n_estimators=10000,n_jobs=-1,subsample=.5)
-cross_val_score(xgbc,x_tf,df.Class,scoring='neg_log_loss')

array([ 7.41845683,  8.36679249,  6.13861137])

In [8]:
cv = CountVectorizer(analyzer=return_same,max_df=.9,
                                   min_df=50)
X_count = cv.fit_transform(df.processed)
print('done transforming data')
xgbc = XGBClassifier(n_estimators=100,n_jobs=-1,subsample=.5)
-cross_val_score(xgbc,X_count,df.Class,scoring='neg_log_loss',n_jobs=1)

done transforming data


array([ 1.76906175,  1.78903816,  1.60486485])

In [14]:
cv = CountVectorizer(analyzer=return_same,max_df=.9,
                                   min_df=100)
X_count = cv.fit_transform(df.processed)
print('done transforming data')
xgbc = XGBClassifier(n_estimators=2000,max_depth=2,learning_rate=.01,n_jobs=-1,subsample=.5)
-cross_val_score(xgbc,X_count,df.Class,scoring='neg_log_loss',n_jobs=1)

done transforming data


array([ 1.81373625,  1.82292181,  1.62473596])

In [19]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_count,df.Class)
xgbc.fit(x_train,y_train)
pred = xgbc.predict(x_test)
confusion_matrix(y_test,pred)

array([[105,   1,   3,  18,  11,   4,  10,   0,   0],
       [  3,  42,   0,   6,   3,   5,  49,   0,   0],
       [  1,   0,   8,   7,   1,   0,   2,   0,   0],
       [ 33,   1,   5, 124,   2,   1,  15,   0,   0],
       [ 13,   4,   2,   7,  26,   4,   7,   0,   0],
       [  5,   3,   0,   6,   2,  39,   9,   0,   0],
       [  2,  29,   8,   5,   3,   0, 178,   0,   2],
       [  0,   1,   0,   1,   0,   0,   2,   0,   1],
       [  1,   1,   0,   0,   0,   0,   0,   0,  10]])

In [21]:
cv = CountVectorizer(analyzer=return_same,max_df=.9,
                                   min_df=100)
X_count = cv.fit_transform(df.processed)
print('done transforming data')
xgbc = XGBClassifier(n_estimators=2000,max_depth=4,learning_rate=.01,n_jobs=-1,subsample=.5)
-cross_val_score(xgbc,X_count,df.Class,scoring='neg_log_loss',n_jobs=1)

done transforming data


array([ 2.00391501,  2.23833243,  1.76744794])

In [22]:
cv = CountVectorizer(analyzer=return_same,max_df=.9,
                                   min_df=300)
X_count = cv.fit_transform(df.processed)
print('done transforming data')
xgbc = XGBClassifier(n_estimators=2000,max_depth=4,learning_rate=.01,n_jobs=-1,subsample=.5)
-cross_val_score(xgbc,X_count,df.Class,scoring='neg_log_loss',n_jobs=1)

done transforming data


array([ 2.00598142,  2.23353243,  1.76561134])

In [28]:
from sklearn.ensemble import RandomForestClassifier

print('done transforming data')
rf = RandomForestClassifier(n_estimators=2000,n_jobs=-1,oob_score=True,)
-cross_val_score(rf,X_count,df.Class,scoring='neg_log_loss',n_jobs=1)

done transforming data


array([ 2.57763235,  2.48680866,  2.36680781])

In [29]:
from sklearn.ensemble import RandomForestClassifier

print('done transforming data')
rf = RandomForestClassifier(n_estimators=2000,n_jobs=-1,oob_score=True,class_weight='balanced')
-cross_val_score(rf,X_count,df.Class,scoring='neg_log_loss',n_jobs=1)

done transforming data


array([ 2.56689829,  2.61658775,  2.47231294])

In [30]:
from sklearn.ensemble import RandomForestClassifier

print('done transforming data')
rf = RandomForestClassifier(n_estimators=500,n_jobs=-1,oob_score=True,class_weight='balanced')
-cross_val_score(rf,X_count,df.Class,scoring='neg_log_loss',n_jobs=1)

done transforming data


array([ 2.831808  ,  3.03867834,  2.73497659])

In [31]:
from sklearn.ensemble import RandomForestClassifier

print('done transforming data')
rf = RandomForestClassifier(n_estimators=5000,n_jobs=-1)
-cross_val_score(rf,X_count,df.Class,scoring='neg_log_loss',n_jobs=1)

done transforming data


array([ 2.32510208,  2.33808567,  2.34031354])

In [33]:
xgbc = XGBClassifier(n_estimators=1000,max_depth=4,learning_rate=.1,n_jobs=-1,subsample=.5)
print(-cross_val_score(xgbc,X_count,df.Class,scoring='neg_log_loss',n_jobs=1).mean())
xgbc = XGBClassifier(n_estimators=1000,max_depth=4,learning_rate=.1,n_jobs=-1,subsample=.25)
print(-cross_val_score(xgbc,X_count,df.Class,scoring='neg_log_loss',n_jobs=1).mean())
xgbc = XGBClassifier(n_estimators=2000,max_depth=2,learning_rate=.01,n_jobs=-1,subsample=.5)
print(-cross_val_score(xgbc,X_count,df.Class,scoring='neg_log_loss',n_jobs=1).mean())

KeyboardInterrupt: 